In [1]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = 'SNP'

import pandas as pd
pd.options.display.max_columns=None

# Add `src` to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path
from pathlib import Path
if 'src' not in Path.cwd().parts:
    src_path = str(Path(get_path('purelib')) / '_src.pth')
    with open(src_path, 'w') as f:
        f.write(str(Path.cwd() / 'src\n'))

# Start the Jupyter loop
from ib_insync import util
util.startLoop()

In [2]:
# Imports 
from utils import Vars, get_pickle

# Set the root
from from_root import from_root
ROOT = from_root()

_vars = Vars(MARKET)
PORT = port = _vars.PORT
OPT_COLS = _vars.OPT_COLS[0]
DATAPATH = ROOT / 'data' / MARKET.lower()


# TODO list
## NSE
- [x] NSE site csv extract
- [x] Reconstruct lots for NSE
- [ ] Correct `get_lots` function
- [ ] Add group_type for NSE margin calc
- [ ] Margin calc formula based on NSE table and und_price

## SNP
- [ ] Margin calc forumula based on IBKR table

## General
- [ ] Correct chain making to include lots and margins in df_chains
- [ ] Correct build base to have default lots and margins



# Correct `get_lots` function

# Make margins for `unds`

In [ ]:
# imports
import asyncio
import json

from utils import get_lots, get_mkt_prices, get_pickle, get_nse_payload

In [ ]:
# Get und_prices
unds = get_pickle(DATAPATH / 'unds.pkl')
df_unds = pd.DataFrame.from_dict(unds.items())
df_unds.columns = ['symbol', 'contract']

# df_und_prices = asyncio.run(get_mkt_prices(port=PORT, contracts = df_unds.contract))

In [ ]:
# url = 'https://www.nseindia.com/api/equity-stockIndices?index=SECURITIES%20IN%20F%26O' # fno list
# url = 'https://nsearchives.nseindia.com/content/nsccl/C_CATG.T01'


# Check out `margins`

## For Options

In [ ]:
from utils import get_pickle, clean_ib_util_df

# Get sample option contracts
df_opt = get_pickle(DATAPATH / 'df_opt_margins.pkl')
cond = df_opt.margin.isnull() | (df_opt.margin == 0)
df_opt = df_opt[cond].sample(80)


In [ ]:
from ib_insync import MarketOrder, IB
from utils import get_lots, chunk_me
from tqdm.asyncio import tqdm
import asyncio

# inputs
df = df_opt
lots_path = None
desc = "Getting Margins"


In [ ]:
lots = df.contract.apply(lambda x: get_lots(contract=x, lots_path=lots_path))
df.order = [MarketOrder(action='SELL', totalQuantity=lot) for lot in lots]
cos = list(zip(df.contract, df.order))
chunks = chunk_me(data=cos, size=25)


In [ ]:
c, o = cos[0]
clean_ib_util_df([c])

In [ ]:
async def get_margins(port: int, chunks: list, desc: str):

    results = list()

    async def wif(c, o):
        df = util.df([c])
        margin = await ib.whatIfOrderAsync(c, o)
        
        return {conId: margin}
    
    for cos in tqdm(chunks, desc=desc):
        tasks = set()
        with await IB().connectAsync(port=port) as ib:
            for c, o in cos:
                tasks.add(wif(c, o))
            res = await asyncio.gather(*tasks)
            results += res

    return results
        

In [ ]:
output = asyncio.run(get_margins(port=PORT, chunks=chunks, desc=desc))

In [ ]:
output

In [ ]:
from utils import clean_a_margin
